In [5]:
!pip install pandas numpy matplotlib seaborn scikit-learn tensorflow optuna



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import optuna
import joblib

In [ ]:
# Carga el conjunto de datos
url = 'housing.csv'
data = pd.read_csv(url)
data["ocean_proximity"] = (data["ocean_proximity"] == "NEAR BAY").astype(int)

In [ ]:
# Análisis Exploratorio de Datos (EDA)
data.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
# Correlaciones
correlation_matrix = data.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Outliers - Tratamiento según sea necesario

# Valores Faltantes
missing_values = data.isnull().sum()
print(missing_values)

In [ ]:
# Tipos de Datos
data.info()


In [ ]:
# Ingeniería/Selección de Características + Limpieza de Datos
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

X_train = train_data.drop("median_house_value", axis=1)
y_train = train_data["median_house_value"].copy()

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(test_data.drop("median_house_value", axis=1))

scaler = StandardScaler()
X_train_scaled_imputed = scaler.fit_transform(X_train_imputed)
X_test_scaled_imputed = scaler.transform(X_test_imputed)


In [ ]:
# Guardar datos preprocesados (puedes almacenarlos en un Feature Store)
np.save('X_train.npy', X_train_scaled_imputed)
np.save('X_test.npy', X_test_scaled_imputed)
np.save('y_train.npy', y_train)